# Scraper para obtener datos públicos

In [155]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup

## Conexión al navegador

In [162]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://nominatransparente.rhnet.gob.mx/nomina-APF'
driver.get(start_url)

## Lectura de archivo csv para obtener nombres

In [167]:
df = pd.read_csv('ejemplo.csv') 
df['NombreCompleto'] = df.apply(lambda row: f"{row['Nombre']} {row['Paterno']} {row['Materno']}", axis=1)
df

,Nombre,Paterno,Materno,NombreCompleto
0,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
1,Luis,Cruz,Jaramillo,Luis Cruz Jaramillo
2,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
3,José Alberto,Montán,López,José Alberto Montán López
4,Andres Manuel,Lopez,Obrador,Andres Manuel Lopez Obrador
5,Juan de Dios,Olivares,Jimenez,Juan de Dios Olivares Jimenez
6,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
7,José Alberto,Montán,López,José Alberto Montán López


In [168]:
data = [ ]

for i in df['NombreCompleto']:
    print(i)
    #WebDriverWait(driver, 10)
    nombre = driver.find_element(By.XPATH, '//*[@id="apfName"]')
    nombre.send_keys(i)
    #WebDriverWait(driver, 10)
    buscar = driver.find_element(By.XPATH, '/html/body/app-root/app-apf/div[2]/div[1]/div[3]/div/div[3]/a')
    #WebDriverWait(driver, 10)
    buscar.click()
    
    time.sleep(5)
    # Obtener identificadores de ventanas
    ventanas = driver.window_handles
    
    # Cambiar al controlador de la ventana emergente
    
    try:
    # Buscar y hacer clic en el botón dentro de la ventana emergente
        driver.switch_to.window(ventanas[-1])
        ok_button = driver.find_element(By.CSS_SELECTOR, 'button.swal2-confirm.swal2-styled')
        ok_button.click()
        print('Se encontró el botón')
    except NoSuchElementException:
        html_table = driver.page_source
    
        # Crear un objeto BeatifulSoup para analizar el html
        soup = BeautifulSoup(html_table, 'html.parser')
        
        # Encontrar todas las finlas de la tabla
        rows = soup.find_all('tr')
        
        for row in rows:
            cells = row.find_all('td')
            # Extraer el texto de cada celda y agregarlo a la lista de datos 
            row_data = [cell.get_text(strip=True) for cell in cells]
            row_data = [cell.split(":")[1].strip() for cell in row_data if ":" in cell]
            data.append(row_data)
        nombre.clear()
        
        pass

    
        
df = pd.DataFrame(data)
    
# Asignar nombres a las columnas
df.columns = ['Número', 'Nombre', 'Institución', 'Puesto', 'Sueldo_bruto_mensual', 'Sueldo_neto_estimado' ]
df.dropna(inplace=True)
df

    
   
    
    

    # Ingresar nombre
    

Bryan Antonio Polito Palma
Se encontró el botón
Luis Cruz Jaramillo
Se encontró el botón
Bryan Antonio Polito Palma
Se encontró el botón
José Alberto Montán López
Se encontró el botón
Andres Manuel Lopez Obrador
Juan de Dios  Olivares Jimenez
Bryan Antonio Polito Palma
Se encontró el botón
José Alberto Montán López
Se encontró el botón


,Número,Nombre,Institución,Puesto,Sueldo_bruto_mensual,Sueldo_neto_estimado
1,1,ANDRES MANUEL LOPEZ OBRADOR,OFICINA DE LA PRESIDENCIA DE LA REPÚBLICA,PRESIDENTE CONSTITUCIONAL DE LOS ESTADOS UNIDO...,"$174,026.00","$120,948.07"
3,1,JUAN DE DIOS OLIVARES JIMENEZ,INSTITUTO POLITÉCNICO NACIONAL,ASISTENTE EJECUTIVO A,"$10,535.76","$8,428.61"
